# Action2

In [1]:
from pprint import pprint

In [2]:
import pulp

In [3]:
import numpy as np

##### 设置单产收益：4种农作物，6个种植地，每一行代表单个农作物分别在六个种植地的收益

In [4]:
costs = np.array([[500, 550, 630, 1000, 800, 700],
                  [800, 700, 600, 950, 900, 930],
                  [1000, 960, 840, 650, 600, 700],
                  [1200, 1040, 980, 860, 880, 780]])

##### 计划播种面积，每一列代表每个农作物的计划播种面积

In [5]:
max_plan_area = [76, 88, 96, 40]

##### 地块面积，每一列代表每块地的最大面积，一共300亩

In [6]:
max_area = [42, 56, 44, 39, 60, 59]

##### 定义规划方程

In [11]:
def farm_program(costs, x_max, y_max):
    row = len(costs)
    col = len(costs[0])
    print(f'row:{row} col:{col}')
    
    # 定义初始问题，这是一个线性规划求最大值的问题
    prob = pulp.LpProblem('Farming', sense=pulp.LpMaximize)
    
    # 定义决策变量，赋予变量种植地的收益，并且是整数类型
    var = [[pulp.LpVariable(f'x{i}{j}', lowBound=0, cat=pulp.LpInteger) for j in range(col)] for i in range(row)]
    
    # 递归展开列表
    flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]
    print('var=', var)
    print('flatten=', flatten)
    print('costs.flatten()', costs.flatten())
    
    # 定义目标函数 （单价* {ij}运量）
    prob += pulp.lpDot(flatten(var), costs.flatten())
    
    # 定义约束条件
    for i in range(row):
        #对于每个农作物来说，种植面积不能超过它的计划种植面积
        prob += pulp.lpSum(var[i]) <= x_max[i]
        
    for j in range(col):
        #对于所有农作物在某块地上的种植不能超过那块地的最大值
        prob += pulp.lpSum([var[i][j] for i in range(row)]) <= y_max[j]
    
    #求解这个规划方程
    prob.solve()
    
    #得到最终的结果
    result = {'objective':pulp.value(prob.objective), \
              'var': [[pulp.value(var[i][j]) for j in range(col)] for i in range(row)]}
    return result


##### 使用上面定义的线性规划方程求解

In [12]:
result = farm_program(costs, max_plan_area, max_area)

row:4 col:6
var= [[x00, x01, x02, x03, x04, x05], [x10, x11, x12, x13, x14, x15], [x20, x21, x22, x23, x24, x25], [x30, x31, x32, x33, x34, x35]]
flatten= <function farm_program.<locals>.<lambda> at 0x000001C928662A60>
costs.flatten() [ 500  550  630 1000  800  700  800  700  600  950  900  930 1000  960
  840  650  600  700 1200 1040  980  860  880  780]


In [9]:
# 输出结果
print(f'最大的收益为{result["objective"]}')

最大的收益为284230.0


In [10]:
print('每个农作物在六块地上的种植面积为：')
pprint(result['var'])

每个农作物在六块地上的种植面积为：
[[0.0, 0.0, 6.0, 39.0, 31.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 29.0, 59.0],
 [2.0, 56.0, 38.0, 0.0, 0.0, 0.0],
 [40.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
